# 第二章：大模型基础原理

## 1. 注意力机制 (Attention Mechanism)

**核心思想**：让模型关注到对当前任务最有帮助的内容

注意力机制允许模型在处理序列数据时，动态地关注不同位置的信息，从而更好地理解上下文关系。

---

## 2. Transformer 架构

**核心特点**：仅仅依赖注意力机制来建模序列之间的全局依赖关系

### 2.1 关键组件

- **Q (Query) 向量**：查询向量，表示"我要找什么"
- **K (Key) 向量**：键向量，表示"我是什么"
- **V (Value) 向量**：值向量，表示"我的内容是什么"

### 2.2 工作原理

通过计算 Q、K、V 之间的权重关系，模型能够：
- 捕捉长距离依赖
- 并行处理序列
- 提高训练效率

---

## 3. 文本生成原理

### 3.1 Token（词元）

**定义**：词汇表中最小的单元

- **Tokenize（分词）**：将完整单词分解到更小的部分
  - 常用工具：spaCy、BERT tokenizer 等
  - 目的：提高模型对词汇的理解和处理能力

### 3.2 自回归生成 (Autoregressive Generation)

**工作原理**：
1. LLM 一次只预测一个 token
2. 根据已有的 token 序列来计算下一个词的概率分布
3. 一般情况下选择概率最高的词
4. 过程是迭代重复的，直到达到某个停止条件

**停止条件**：
- 理想情况下由模型决定
- 输出一个结束序列标记（EOS - End of Sequence）
- 或达到最大生成长度限制

### 3.3 生成过程的干预

#### 3.3.1 Temperature（温度参数）

**作用**：通过温度值来控制生成结果的随机性

- **温度越高**：概率分布越平滑，随机性增加，输出更多样化
- **温度越低**：概率分布越尖锐，随机性降低，输出更确定

**典型取值**：
- `temperature = 0.1-0.3`：更确定、更保守的输出
- `temperature = 0.7-0.9`：平衡创造性和准确性
- `temperature > 1.0`：更随机、更创造性的输出

#### 3.3.2 采样策略

**1. 贪心策略 (Greedy Strategy)**
- 始终选择概率最高的单词
- **优点**：快速、确定性强
- **缺点**：可能陷入重复循环，缺乏多样性

**2. 波束搜索 (Beam Search)**
- 每个时间步保留最可能的 `num_beams` 个词
- 从中选择概率最高的序列
- **优点**：在确定性和多样性之间取得平衡
- **缺点**：计算成本较高

**3. Top-K 采样**
- 从概率最高的 K 个词中随机选择下一个词
- **优点**：简单有效，控制候选词数量
- **适用场景**：需要一定随机性的生成任务

**4. Top-P（核采样，Nucleus Sampling）**
- 从概率累积达到 P 的最小词集中随机选择下一个词
- 动态调整采样范围
- **优点**：自适应选择候选词数量，更灵活
- **适用场景**：需要平衡质量和多样性的任务

#### 3.3.3 生成策略建议

| 场景 | 推荐策略 | 参数设置 |
|------|---------|---------|
| **创造性任务**<br>（写诗、创意写作） | Top-P 或 Top-K | `temperature=0.9-1.2`<br>`top_p=0.9-0.95` |
| **准确性任务**<br>（翻译、摘要） | 贪心或波束搜索 | `temperature=0.1-0.3`<br>`num_beams=3-5` |
| **平衡任务**<br>（对话、问答） | Top-P | `temperature=0.7-0.9`<br>`top_p=0.9` |

---

## 4. 核心要点总结

### 4.1 注意力机制
- ✅ 让模型关注重要信息
- ✅ 捕捉长距离依赖关系
- ✅ 提高模型理解能力

### 4.2 Transformer 架构
- ✅ 基于注意力机制
- ✅ Q、K、V 向量是关键
- ✅ 支持并行处理

### 4.3 文本生成
- ✅ Token 是最小单元
- ✅ 自回归逐词生成
- ✅ 可通过参数控制生成质量

### 4.4 参数调优
- ✅ Temperature 控制随机性
- ✅ 采样策略影响输出质量
- ✅ 根据任务选择合适的策略

---



---

## 5. 实践代码

### 5.1 环境准备与依赖安装

在使用开源大语言模型之前，首先需要搭建合适的开发环境并安装所需的依赖库。

**步骤 1：安装 PyTorch**

根据你的硬件环境（CPU 或 GPU）选择合适的深度学习框架：

- **CPU 版本**：

In [7]:
# 安装 PyTorch (CPU 版本)
# !pip install torch

In [8]:
# 安装 PyTorch (GPU 版本，CUDA 11.8)
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

**步骤 2：验证安装**

安装完成后，可以通过以下代码验证安装是否成功：

In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available: {torch.cuda.is_available()}")

# 如果 CUDA 可用，说明你的 GPU 环境已经配置成功，可以加速模型推理

PyTorch version: 2.5.1+cu121
Is CUDA available: True


In [8]:
# 安装/升级 transformers 库
# 如果遇到 GenerationMixin 导入错误，请运行以下命令升级 transformers：
!pip install --upgrade transformers>=4.33.0
# 或者直接安装最新版本：
# !pip install --upgrade transformers

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 指定模型名称或本地路径
# 注意：这里使用本地路径，如果使用 Hugging Face Hub 上的模型，直接使用模型名称即可
# 例如：model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model_name = "Qwen/Qwen2.5-0.5B-Instruct"  # 请根据实际情况修改路径

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 将模型移动到 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"Model loaded on {device}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Anaconda\envs\hands-on-aai\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Peiyan Li\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model loaded on cuda


In [3]:
# 查看模型参数量
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params / 1e6:.2f}M")

# 查看模型结构（可选，输出较长）
print(model)

Total model parameters: 494.03M
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), ep

---

### 5.3 文字续写任务

文字续写是生成式大语言模型的基本功能之一。以下是详细的代码示例：

In [4]:
from transformers import pipeline

# 创建文本生成管道
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device  # 指定设备
)

# 输入提示文本
prompt = "从前有一个小村庄，村里住着一位聪明的老人。"

# 生成续写文本
generated_text = text_generator(
    prompt,
    max_length=200,  # 最大生成长度
    min_length=50,   # 最小生成长度
    do_sample=True,  # 是否使用采样
    early_stopping=True  # 提前停止生成
)[0]['generated_text']

print("生成的文本：")
print(generated_text)

Device set to use cuda
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


生成的文本：
从前有一个小村庄，村里住着一位聪明的老人。有一天，村里的孩子都问他一个问题：“你是什么？”
老人们说：“我是谁？”
孩子们疑惑地问道：“我们都是谁？”

老人继续说道：“我就是你所说的‘我是谁’。”
孩子们的困惑和不解让他更加感到无奈。
于是，他开始向孩子们讲述他的故事，告诉他们自己的名字、家庭住址以及他的人生经历。
孩子们被这个充满智慧的故事深深吸引，纷纷询问他更多的信息。

经过一段时间后，孩子们逐渐对这位老人生出了敬佩之情。在一次外出游玩时，孩子们发现了一个神秘的石碑，上面刻有“我是谁”的字样。他们带着疑问去拜访了那位老人，老人也耐心地回答了他们的问题。

最终，在一个阳光明媚的日子里，孩子们决定邀请这位老人回家过夜，一起分享他们心中的秘密。

从此以后，每当孩子们遇到困难或需要帮助时，他们都会想起那个充满智慧的老人和他的故事。而这位老人也成为了村里流传最广的英雄人物之一。他的故事激励着每一个孩子，让他们知道只要勇于探索未知，就一定能够找到属于自己的光芒。


---

### 5.4 生成过程的干预实践

在生成式大语言模型的生成过程中，可以通过调整参数来干预生成结果的风格和质量。这些参数主要包括温度（Temperature）和采样策略。

**示例：对比不同参数设置下的生成效果**

In [5]:
# 输入提示文本
prompt =  "请介绍一下大语言模型的注意力机制。"

# 生成严谨回答（低温值，贪心采样）
rigorous_text = text_generator(
    prompt,
    max_length=200,
    temperature=0.1,  # 低温值
    top_k=10,          # Top-k 采样，低k值
    top_p=0.5,        # Top-p 采样， 低p值
    num_return_sequences=1,
    repetition_penalty=1.2  # 避免重复
)[0]['generated_text']

print("-"*50)

print("严谨回答：")
print(rigorous_text[len(prompt):])  # 去掉提示文本部分

# 生成创意回答（高温值，Top-p 采样）
creative_text = text_generator(
    prompt,
    max_length=200,
    temperature=1.5,  # 高温值
    top_k=50,         # Top-k 采样， 高k值
    top_p=0.9,        # Top-p 采样，高p值
    num_return_sequences=1,
    repetition_penalty=1.0  # 允许一定重复
)[0]['generated_text'][len(prompt):]  # 去掉提示文本部分

print("\n创意回答：")
print(creative_text)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------
严谨回答：
 大规模预训练的语言模型，如GPT、BERT等，在处理文本时会使用一种称为“注意力机制”的技术来增强其性能和理解能力。

在传统的自然语言处理中，我们通常关注的是如何让计算机理解和生成人类语言的能力最大化。然而，随着深度学习的发展，人们发现通过引入注意力机制可以显著提高机器对输入信息的理解能力和表达质量。

### 注意力机制的基本概念

1. **注意力过程**：这是指一个特定任务（例如识别某个单词或句子）与另一个部分之间的交互作用。
2. **权重分配**：每个位置上的词被赋予不同的权重，这些权重反映了该位置上词汇的重要性。
3. **局部视图**：在一个给定的位置上，所有可能影响当前上下文的信息都被考虑进来，并且它们会被加权并进行组合以形成最终的结果。

### 为什么需要注意力机制？

- **提升理解效率**: 在大规模数据集的学习过程中，由于样本数量庞大，传统方法难以有效捕捉全局特征。而注意力机制能够帮助模型更好地聚焦于关键点，从而更高效地完成复杂的分析任务。
  
- **减少过拟合风险**: 当前许多大型语言模型存在过度拟合的问题，即模型容易

创意回答：
 一般而言，大语言模型（如BERT和GPT）并没有在传统上下文中理解问题（例如，在询问机器中回答的上下文），因此，在这些模型中并没有处理注意力机制。也就是说，它们仅仅具有处理其他信息，如语言模型和数据相关问题的性能。

但是，有许多现代的机器学习方法和技术可能使用注意力机制。例如，在图神经网络和LSTM神经网络上，通过在特定部分或对特定位置的输入上应用自回归训练，可以在这些模型中增强语言理解能力，尤其是在处理词频聚类的问题和对多词句的识别或命名时。这种训练过程可能不需要注意之前的信息的完整性，并且会考虑到所有相邻输入的可能性。

此外，有些机器学习方法包括多模态和混合模态，它们能够处理由不同模态输入的情境和情景。尽管这些学习过程目前没有提供直接的注意力机制，但它可能被用来增强机器的多模态能力和理解问题。

总之，尽管现代的语言理解和生成模型（如BERT和GPT）可以处理由具体语言表达、句子和多语料描述的信息，没有专门的注意力机制来针对这些特定问题，但某些领域的机器学习研究正在探索如何使

可以看到，严谨回答分点清晰，显得专业可信；创意回答更偏向平铺直叙，显得通俗易懂。

---

### 5.5 本地化智能体

基于开源模型，可以构建本地化的智能体，用于单轮或多轮对话。

#### 5.5.1 单轮对话：问答助手

单轮对话系统用于回答用户提出的问题，适合简单的问答场景。

In [6]:
def single_turn_qa(question):
    # 输入问题
    prompt = f"问题：{question}\n回答："
    
    # 生成回答
    response = text_generator(
        prompt,
        max_length=300,
        temperature=0.5,  # 中等温度值
        top_k=50,
        top_p=0.9,
        num_return_sequences=1,
        repetition_penalty=1.1
    )[0]['generated_text'].split("回答：")[1]
    
    return response

# 测试单轮问答
question = "什么是人工智能？"
answer = single_turn_qa(question)
print(f"问题：{question}")
print(f"回答：{answer}")

Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


问题：什么是人工智能？
回答：人工智能（Artificial Intelligence，简称AI）是指由计算机系统所表现出的智能行为或功能。它包括机器学习、深度学习等技术，并且是研究和开发用于模拟、延伸和扩展人类智能的一门学科。

人工智能在许多领域都有广泛的应用，如语音识别、图像处理、自然语言处理、自动驾驶汽车、医疗诊断、金融分析等领域。随着技术的发展，人工智能正逐渐渗透到我们的生活中，成为我们生活的一部分。然而，人工智能也面临着一些挑战和争议，例如隐私保护、伦理道德等问题需要我们在使用人工智能时进行深入思考和讨论。总的来说，人工智能是一个充满活力和潜力的领域，但同时也需要我们谨慎对待其发展带来的影响。


#### 5.5.2 多轮对话：聊天机器人

多轮对话系统能够维护对话历史，使对话更具连贯性和上下文感知能力。

In [7]:
class MultiTurnChatbot:
    def __init__(self):
        self.history = []  # 存储对话历史
    
    def generate_response(self, user_input):
        # 构建包含历史的提示
        prompt = "对话历史：\n"
        for turn in self.history:
            prompt += f"用户：{turn['user']}\n助手：{turn['assistant']}\n"
        prompt += f"用户：{user_input}\n助手："
        
        # 生成回答
        response = text_generator(
            prompt,
            max_length=300,
            temperature=0.5,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=1.1
        )[0]['generated_text'].split("助手：")[-1]
        
        # 保存当前对话到历史
        self.history.append({"user": user_input, "assistant": response})
        
        return response

# 测试多轮对话
chatbot = MultiTurnChatbot()

# 第一轮对话
user_input = "什么是人工智能。"
response = chatbot.generate_response(user_input)
print("----------------对话1----------------")
print(f"用户：{user_input}")
print(f"助手：{response}")

# 第二轮对话
user_input = "列举它的应用场景。"
response = chatbot.generate_response(user_input)
print("----------------对话2----------------")
print(f"用户：{user_input}")
print(f"助手：{response}")

# 第三轮对话
user_input = "使用人工智能需要注意什么。"
response = chatbot.generate_response(user_input)
print("----------------对话3----------------")
print(f"用户：{user_input}")
print(f"助手：{response}")

Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


----------------对话1----------------
用户：什么是人工智能。
助手：我认为，随着技术的不断进步，人工智能将会变得更加智能化、自主化和人性化。它将更加擅长处理复杂的任务，甚至有可能实现自我学习和自我改进


Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


----------------对话2----------------
用户：列举它的应用场景。
助手：我认为，人工智能将在未来的几年内取得更大的发展，并且会朝着更广泛的应用领域拓展。例如，我们可以期待看到更多的机器人、智能音箱、智能家居设备等产品出现；我们也可以预期到更多的人工智能应用将被开发出来，以提高工作效率和生活质量；同时，我们也需要关注如何确保人工智能的安全性和可靠性，防止其滥用或误用。总的来说，我认为人工智能是未来科技发展的主要方向之一，具有巨大的发展潜力和潜力。
----------------对话3----------------
用户：使用人工智能需要注意什么。
助手：我建议在使用人工智能时要遵守相关的法律法规，保护个人隐私，避免过度依赖人工智能而忽视了人的价值和意义。此外，也要注重人工智能的发展过程中的伦理问题，确保其对社会的影响是积极有益的。最后，我们应该鼓励和支持人工智能的研究和发展，让其为人类带来更多的便利和福祉。总之，我认为人工智能是一个既充满机遇又面临挑战的重要领域，我们需要做好充分准备并加以利用。


> 💡 **提示**：在多轮对话中，对话历史的维护是关键。通过将多轮对话拼接作为模型的输入，模型能够根据之前的对话内容生成更连贯的回答。你可以根据实际需求调整对话历史的长度，以平衡模型的推理速度和对话连贯性。

---

## 6. 代码要点总结

### 6.1 环境配置
- ✅ 安装 PyTorch（根据硬件选择 CPU 或 GPU 版本）
- ✅ 安装 Transformers 库
- ✅ 验证 CUDA 是否可用（GPU 加速）

### 6.2 模型加载
- ✅ 使用 `AutoTokenizer` 加载分词器
- ✅ 使用 `AutoModelForCausalLM` 加载预训练模型
- ✅ 将模型移动到合适的设备（CPU/GPU）

### 6.3 文本生成
- ✅ 使用 `pipeline` 创建文本生成管道
- ✅ 通过 `max_length`、`min_length` 控制生成长度
- ✅ 通过 `temperature`、`top_k`、`top_p` 控制生成质量

### 6.4 参数调优
- ✅ **严谨回答**：`temperature=0.1`, `top_k=10`, `top_p=0.5`
- ✅ **创意回答**：`temperature=1.5`, `top_k=50`, `top_p=0.9`
- ✅ **平衡回答**：`temperature=0.5-0.7`, `top_p=0.9`

### 6.5 智能体构建
- ✅ 单轮问答：简单直接，适合一次性问题
- ✅ 多轮对话：维护历史，提供上下文感知

---

## 7. 注意事项

1. **模型路径**：确保模型路径正确，或使用 Hugging Face Hub 上的模型名称
2. **内存管理**：大模型需要较多内存，注意监控系统资源
3. **参数调优**：根据具体任务调整生成参数，平衡质量和多样性
4. **对话历史**：多轮对话中注意控制历史长度，避免超出模型上下文窗口

---

**下一章**：我们将学习如何在实际应用中调用和使用大模型 API。